In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
## Standard libraries
import os
import numpy as np
import random
import math
import json
from functools import partial

## Imports for plotting
import matplotlib.pyplot as plt
plt.set_cmap('cividis')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## Torchvision
import torchvision
from torchvision.datasets import CIFAR100
from torchvision import transforms

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/tutorial6"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

INFO:lightning_lite.utilities.seed:Global seed set to 42


Device: cpu


In [4]:
import numpy as np
import torch
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from sklearn import preprocessing
from enum import Enum 
import copy
# import SGD for optimizer
from torch.optim import SGD

# import Adam for optimizer
from torch.optim import Adam

# to measure the performance import L1Loss
from torch.nn import L1Loss

In [5]:
training_x = np.load('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework3/training_x.npy')

In [6]:
training_x

array([[14.33333333,  0.33333333,  7.66666667, ...,  2.        ,
         0.        ,  0.33333333],
       [14.        ,  0.33333333,  9.33333333, ...,  2.        ,
        -0.33333333,  0.33333333],
       [14.        ,  0.33333333,  5.66666667, ...,  2.        ,
         0.        ,  0.66666667],
       ...,
       [16.        ,  0.        ,  2.        , ...,  1.        ,
         0.        ,  3.        ],
       [16.        ,  0.        ,  1.        , ...,  2.        ,
         0.66666667,  4.66666667],
       [15.        ,  0.        ,  0.66666667, ...,  2.        ,
         1.33333333,  4.33333333]])

In [59]:
new_training_x = []


In [60]:
new_training_x 

[]

In [61]:
for i in range(8,training_x.shape[0]-8):
  y = []
  for j in range(8):
    a = training_x[i+j-8]
    k = a.tolist()
    y.append(k)
    # print(y)
  new_training_x.append(y)

In [14]:
# new_training_x

In [76]:
# new_training_x[0]

In [63]:
len(new_training_x)


8744

In [17]:
training_y = np.load('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework3/training_y.npy')

In [77]:
# training_y[:17]

In [92]:
new_training_y = []

In [93]:
for i in range(8,training_x.shape[0]-8):
  y = []
  for j in range(8):
    a = training_y[i+j][0]
    k = a.tolist()
    y.append(k)
    # print(y)
  new_training_y.append(y)

In [94]:
new_training_y[0]

[38.0, 34.0, 33.0, 31.0, 30.0, 33.0, 37.0, 36.0]

In [23]:
testing_x = np.load('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework3/testing_x.npy')

In [24]:
testing_x 

array([[15.        ,  0.        ,  0.66666667, ...,  2.        ,
         1.33333333,  3.33333333],
       [14.66666667,  0.        ,  0.66666667, ...,  2.        ,
         0.66666667,  3.33333333],
       [14.        ,  0.        ,  0.66666667, ...,  2.        ,
         1.        ,  3.        ],
       ...,
       [16.        ,  0.        ,  0.66666667, ...,  1.        ,
         0.33333333,  3.33333333],
       [16.        ,  0.        ,  1.        , ...,  1.        ,
         0.33333333,  3.        ],
       [16.        ,  0.        ,  1.        , ...,  1.        ,
         0.33333333,  4.33333333]])

In [78]:
new_testing_x = []

In [79]:
for i in range(8,testing_x.shape[0]-8):
  y = []
  for j in range(8):
    a = testing_x[i+j-8]
    k = a.tolist()
    y.append(k)
    # print(y)
  new_testing_x.append(y)

In [82]:
# new_testing_x[0]

In [83]:
class testing_Dataset(Dataset):
    # This loads the data and converts it, make data rdy
    def __init__(self):
        # load data
        self.df= new_testing_x
        # extract labels

        # conver to torch dtypes
        self.dataset=torch.tensor(self.df).float()

    # This returns the total amount of samples in your Dataset
    def __len__(self):
        return len(self.dataset)
    
    # This returns given an index the i-th sample and label
    def __getitem__(self, idx):
        return self.dataset[idx]

In [84]:
testing_dataset =testing_Dataset()
testing_dataset

In [85]:
testing_dataset[0]

tensor([[ 15.0000,   0.0000,   0.6667,  11.0000,  12.6667,  42.0000, 159.3333,
           0.0000,  64.0000,   8.6667,   2.0000,   1.3333,   3.3333],
        [ 14.6667,   0.0000,   0.6667,  11.0000,  12.6667,  41.0000, 161.3333,
           0.0000,  63.6667,   7.3333,   2.0000,   0.6667,   3.3333],
        [ 14.0000,   0.0000,   0.6667,  10.3333,  12.3333,  41.0000, 149.6667,
           0.0000,  64.3333,   6.3333,   2.0000,   1.0000,   3.0000],
        [ 14.0000,   0.0000,   0.6667,   8.0000,   9.0000,  40.3333, 135.6667,
           0.0000,  65.0000,   5.3333,   1.0000,   0.6667,   3.0000],
        [ 14.0000,   0.0000,   0.3333,   7.3333,   8.3333,  39.6667, 119.6667,
           0.0000,  65.3333,   4.3333,   1.0000,   0.6667,   3.0000],
        [ 13.0000,   0.0000,   0.3333,   8.0000,   8.3333,  38.3333, 108.3333,
           0.0000,  64.3333,   4.3333,   1.0000,   0.6667,   2.6667],
        [ 13.0000,   0.0000,   0.6667,  10.0000,  11.0000,  34.6667, 102.3333,
           0.0000,  64.3333

In [86]:
batch_size = 100
testing_loader = DataLoader(testing_dataset, batch_size, num_workers=2, pin_memory=True)

In [88]:
inp_data = testing_loader.dataset[100]
print("Input data:", inp_data)


Input data: tensor([[14.6667,  0.3333, 14.3333, 21.3333, 35.6667, 10.3333, 32.0000,  0.0000,
         85.3333,  1.6667,  2.0000,  0.0000,  0.0000],
        [14.6667,  0.0000, 11.0000, 17.6667, 29.0000, 10.6667, 31.3333,  0.0000,
         85.6667,  1.3333,  2.0000, -0.3333,  0.0000],
        [14.6667,  0.3333, 26.3333, 20.0000, 46.3333,  8.3333, 38.0000,  0.0000,
         86.0000,  2.0000,  3.0000,  0.0000,  0.0000],
        [15.0000,  0.3333, 30.3333, 24.6667, 55.3333,  7.3333, 44.3333,  0.0000,
         84.3333,  4.0000,  2.0000,  0.0000,  0.0000],
        [17.0000,  1.0000, 31.3333, 34.3333, 66.0000,  9.0000, 52.0000,  0.0000,
         78.0000,  5.6667,  2.0000,  0.0000,  0.3333],
        [19.0000,  0.6667, 24.6667, 49.0000, 74.0000, 16.0000, 65.0000,  0.0000,
         67.0000,  5.3333,  2.0000, -0.3333,  0.3333],
        [21.0000,  0.6667, 15.6667, 46.3333, 62.0000, 30.0000, 68.0000,  0.0000,
         59.6667,  4.0000,  2.0000,  0.0000,  0.6667],
        [22.6667,  0.6667,  6.3333, 

In [105]:
class TrainingDataset(Dataset):
    # This loads the data and converts it, make data rdy
    def __init__(self):
        # load data
        self.df= new_training_x[:8000]
        # extract labels
        self.df_labels= new_training_y[:8000]
        # conver to torch dtypes
        self.dataset=torch.tensor(self.df).float()

        self.labels=torch.tensor(self.df_labels).long()
    
    # This returns the total amount of samples in your Dataset
    def __len__(self):
        return len(self.dataset)
    
    # This returns given an index the i-th sample and label
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

In [106]:
class ValidDataset(Dataset):
    # This loads the data and converts it, make data rdy
    def __init__(self):
        # load data
        self.df= new_training_x[8000:]
        # extract labels
        self.df_labels= new_training_y[8000:]
        # conver to torch dtypes
        self.dataset=torch.tensor(self.df).float()

        self.labels=torch.tensor(self.df_labels).long()
    
    # This returns the total amount of samples in your Dataset
    def __len__(self):
        return len(self.dataset)
    
    # This returns given an index the i-th sample and label
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

In [107]:
train_ds = TrainingDataset()
val_ds = ValidDataset()

In [109]:
print(len(train_ds), len(val_ds))

from torch.utils.data import DataLoader

batch_size = 100

train_loader = DataLoader(train_ds,batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size, num_workers=2, pin_memory=True)

8000 744


In [110]:
inp_data, ans = train_loader.dataset[0]
print("Input data:", inp_data)
print("Labels:    ", ans)

Input data: tensor([[14.3333,  0.3333,  7.6667, 37.0000, 44.6667, 12.3333, 53.0000,  0.0000,
         76.3333,  1.6667,  2.0000,  0.0000,  0.3333],
        [14.0000,  0.3333,  9.3333, 34.3333, 43.6667, 14.3333, 53.6667,  0.0000,
         78.0000,  4.6667,  2.0000, -0.3333,  0.3333],
        [14.0000,  0.3333,  5.6667, 36.6667, 42.3333, 14.6667, 55.0000,  0.0000,
         77.3333,  4.0000,  2.0000,  0.0000,  0.6667],
        [13.6667,  0.6667, 16.6667, 37.3333, 54.3333, 12.3333, 57.3333,  0.0000,
         78.6667,  3.0000,  2.0000, -0.3333,  0.3333],
        [13.3333,  0.3333,  6.3333, 24.3333, 30.6667, 21.3333, 54.3333,  0.0000,
         80.3333,  2.0000,  2.0000,  0.3333,  0.6667],
        [13.3333,  0.3333, 12.0000, 24.6667, 37.3333, 22.0000, 49.6667,  0.0000,
         81.0000,  2.6667,  2.0000,  0.0000,  0.6667],
        [13.3333,  0.3333, 28.0000, 27.6667, 55.6667, 21.0000, 49.3333,  0.0000,
         81.6667,  6.6667,  2.0000, -0.3333,  0.3333],
        [13.6667,  0.3333, 22.6667, 

In [111]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [112]:
class MultiheadAttention(nn.Module):

    def __init__(self, input_dim, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Stack all weight matrices 1...h together for efficiency
        # Note that in many implementations you see "bias=False" which is optional
        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, embed_dim)

        self._reset_parameters()

    def _reset_parameters(self):
        # Original Transformer initialization, see PyTorch documentation
        nn.init.xavier_uniform_(self.qkv_proj.weight)
        self.qkv_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.o_proj.weight)
        self.o_proj.bias.data.fill_(0)

    def forward(self, x, mask=None, return_attention=False):
        batch_size, seq_length, _ = x.size()
        qkv = self.qkv_proj(x)

        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)

        # Determine value outputs
        values, attention = scaled_dot_product(q, k, v, mask=mask)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        values = values.reshape(batch_size, seq_length, self.embed_dim)
        o = self.o_proj(values)

        if return_attention:
            return o, attention
        else:
            return o

In [113]:
class EncoderBlock(nn.Module):

    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()

        # Attention layer
        self.self_attn = MultiheadAttention(input_dim, input_dim, num_heads)

        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )

        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Attention part
        attn_out = self.self_attn(x, mask=mask)
        x = x + self.dropout(attn_out)
        x = self.norm1(x)

        # MLP part
        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm2(x)

        return x

In [114]:
class TransformerEncoder(nn.Module):

    def __init__(self, num_layers, **block_args):
        super().__init__()
        self.layers = nn.ModuleList([EncoderBlock(**block_args) for _ in range(num_layers)])

    def forward(self, x, mask=None):
        for l in self.layers:
            x = l(x, mask=mask)
        return x

    def get_attention_maps(self, x, mask=None):
        attention_maps = []
        for l in self.layers:
            _, attn_map = l.self_attn(x, mask=mask, return_attention=True)
            attention_maps.append(attn_map)
            x = l(x)
        return attention_maps

In [115]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model)
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [116]:
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [117]:
class TransformerPredictor(pl.LightningModule):

    def __init__(self, input_dim, model_dim, num_classes, num_heads, num_layers, lr, warmup, max_iters, dropout=0.0, input_dropout=0.0):
        """
        Inputs:
            input_dim - Hidden dimensionality of the input
            model_dim - Hidden dimensionality to use inside the Transformer
            num_classes - Number of classes to predict per sequence element
            num_heads - Number of heads to use in the Multi-Head Attention blocks
            num_layers - Number of encoder blocks to use.
            lr - Learning rate in the optimizer
            warmup - Number of warmup steps. Usually between 50 and 500
            max_iters - Number of maximum iterations the model is trained for. This is needed for the CosineWarmup scheduler
            dropout - Dropout to apply inside the model
            input_dropout - Dropout to apply on the input features
        """
        super().__init__()
        self.save_hyperparameters()
        self._create_model()

    def _create_model(self):
        # Input dim -> Model dim
        self.input_net = nn.Sequential(
            nn.Dropout(self.hparams.input_dropout),
            nn.Linear(self.hparams.input_dim, self.hparams.model_dim)
        )
        # Positional encoding for sequences
        self.positional_encoding = PositionalEncoding(d_model=self.hparams.model_dim)
        # Transformer
        self.transformer = TransformerEncoder(num_layers=self.hparams.num_layers,
                                              input_dim=self.hparams.model_dim,
                                              dim_feedforward=2*self.hparams.model_dim,
                                              num_heads=self.hparams.num_heads,
                                              dropout=self.hparams.dropout)
        # Output classifier per sequence lement
        self.output_net = nn.Sequential(
            nn.Linear(self.hparams.model_dim, self.hparams.model_dim),
            nn.LayerNorm(self.hparams.model_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(self.hparams.dropout),
            nn.Linear(self.hparams.model_dim, self.hparams.num_classes)
        )

    def forward(self, x, mask=None, add_positional_encoding=True):
        """
        Inputs:
            x - Input features of shape [Batch, SeqLen, input_dim]
            mask - Mask to apply on the attention outputs (optional)
            add_positional_encoding - If True, we add the positional encoding to the input.
                                      Might not be desired for some tasks.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        x = self.transformer(x, mask=mask)
        x = self.output_net(x)
        return x

    @torch.no_grad()
    def get_attention_maps(self, x, mask=None, add_positional_encoding=True):
        """
        Function for extracting the attention matrices of the whole Transformer for a single batch.
        Input arguments same as the forward pass.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        attention_maps = self.transformer.get_attention_maps(x, mask=mask)
        return attention_maps

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)

        # Apply lr scheduler per step
        lr_scheduler = CosineWarmupScheduler(optimizer,
                                             warmup=self.hparams.warmup,
                                             max_iters=self.hparams.max_iters)
        return [optimizer], [{'scheduler': lr_scheduler, 'interval': 'step'}]

    def training_step(self, batch, batch_idx):
        raise NotImplementedError

    def validation_step(self, batch, batch_idx):
        raise NotImplementedError

    def test_step(self, batch, batch_idx):
        raise NotImplementedError

In [118]:
class PM2_5_Predictor(TransformerPredictor):

    def _calculate_loss(self, batch, mode="train"):
        # Fetch data and transform categories to one-hot vectors
        inp_data, labels = batch
        inp_data = F.one_hot(inp_data, num_classes=self.hparams.num_classes).float()

        # Perform prediction and calculate loss and accuracy
        preds = self.forward(inp_data, add_positional_encoding=True)
        loss = F.cross_entropy(preds.view(-1,preds.size(-1)), labels.view(-1))
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        # Logging
        self.log(f"{mode}_loss", loss)
        self.log(f"{mode}_acc", acc)
        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, _ = self._calculate_loss(batch, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")

In [119]:
def train_PM2_5(**kwargs):
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "PM2_5_Task")
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=10,
                         gradient_clip_val=5)
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, "PM2_5_Task.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = PM2_5_Predictor.load_from_checkpoint(pretrained_filename)
    else:
        model = PM2_5_Predictor(max_iters=trainer.max_epochs*len(train_loader), **kwargs)
        trainer.fit(model, train_loader, val_loader)

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    test_result = trainer.test(model, testing_loader, verbose=False)
    result = {"val_acc": val_result[0]["test_acc"]}

    model = model.to(device)
    return model, result

In [120]:
PM2_5_model, PM2_5_result = train_PM2_5(input_dim=train_loader.dataset.num_categories,
                                              model_dim=13*2,
                                              num_heads=1,
                                              num_classes=train_loader.dataset.num_categories,
                                              num_layers=1,
                                              dropout=0.0,
                                              lr=5e-4,
                                              warmup=50)

AttributeError: ignored

In [ ]:
print(f"Val accuracy:  {(100.0 * reverse_result['val_acc']):4.2f}%")
print(f"Test accuracy: {(100.0 * reverse_result['test_acc']):4.2f}%")